# Simple RAG sample
RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as search and databases) with the capabilities of generative large language models (LLMs). By combining your data and world knowledge with LLM language skills, grounded generation is more accurate, up-to-date, and relevant to your specific needs.

In [1]:
!pip install google-cloud-aiplatform


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai

In [3]:
PROJECT_ID = "project-1-test-ai"
display_name = "test_corpus"
paths = ["gs://tenant1-local-bucket/live"]  # Supports Google Cloud Storage and Google Drive Links

# Initialize Vertex AI API once per session
vertexai.init(project=PROJECT_ID, location="us-central1")

In [4]:
# Create RagCorpus
# Configure embedding model, for example "text-embedding-004".
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-004"
)

rag_corpus = rag.create_corpus(
    display_name=display_name,
    embedding_model_config=embedding_model_config,
)

# Import Files to the RagCorpus
rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size=512,  # Optional
    chunk_overlap=100,  # Optional
    max_embedding_requests_per_min=900,  # Optional
)

imported_rag_files_count: 12

In [5]:
# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="which are the tools to sell more?",
    similarity_top_k=1,  # Optional
    vector_distance_threshold=0.5,  # Optional
)
print(response)

contexts {
  contexts {
    source_uri: "gs://tenant1-local-bucket/live/[Mensis]_Linkedin_para_generar_citas_de_ventas.txt"
    text: "Que la gente conoce y que la gente va a conocer m\303\241s fuerte cada vez Entonces es importante que este Go que ac\303\241, eh? Pero en resumen creo que solo eso la foto ac\303\241 con el correo que agregu\303\251 es goke, verdad? Para que se vea resumir esto?\nEh, Y este y no Nada m\303\241s, eso estaba bien. Bueno, Yo te agradezco mucho, vamos a hacer un primer remake a ver c\303\263mo nos va y Qu\303\251 ch\303\251vere, no muy buen comienzo Pero cu\303\241ndo d\303\251jame explicarle otro producto que les va a encantar a ambos S\303\255 porque yo tengo aqu\303\255 otra otra preguntica Dale dale la pregunta Dale s\303\255 porque, eh, No s\303\251 si vamos a hablar de citas estrat\303\251gicas o de participaci\303\263n que tambi\303\251n hab\303\255as puesto en tu en tu tus notas al principio, Eh s\303\255. Okay, Entonces vamos a a ver este este ya l

In [6]:
# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            similarity_top_k=3,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)
# Create a gemini-pro model instance
rag_model = GenerativeModel(
    model_name="gemini-1.5-flash-001", tools=[rag_retrieval_tool]
)

In [7]:
# Generate response
response = rag_model.generate_content("which are the tools to sell more?")
print(response.text)

The sources provide information on using LinkedIn for sales and recruiting. These are some of the mentioned tools:

* **LinkedIn profile:** A strong profile is the foundation for success.
* **Sales Navigator:** A tool used by many for finding prospects. 
* **LinkedIn Ads:** Used to reach potential customers and clients.
* **9 Steps of Sales:** This source focuses on building rapport with clients, understanding their needs, and creating an emotional connection. 
* **Effective communication:** Building trust, providing solutions, and identifying needs are vital for successful selling. 
* **Efficient processes:**  Having a clear and agile process saves time and keeps motivation high. 



### References:
- https://cloud.google.com/use-cases/retrieval-augmented-generation?hl=en
- https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-rag-generate-content
- https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-rag-upload-file
- https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-rag-quickstart